In [1]:
import os
import pandas as pd
import shutil

In [47]:
run = 3
num_img_category = 1
img_type = "original"

###########################################################################################

API_NAME = 'grad_cam_'+img_type+'/run_'+str(run)+'/'+str(num_img_category)+'_shot'

heat_map_path = API_NAME
res_path = "./../Test Data/result_in_csv_file/"+img_type+'/run_'+str(run)+'/'+str(num_img_category)+'_shot'
test_data_path = "./../Test Data/test_data_trained_on_"+img_type
out_path = "heatmap_Male_Female_"+img_type+"_run"+str(run)+"_"+str(num_img_category)+'_shot'

def createFolder(path):
    try:
        os.mkdir(path)
    except:
        pass
    
createFolder(out_path)
df_total = pd.DataFrame()
countries = os.listdir(heat_map_path)

for country in countries:
    country_path_heatmap = heat_map_path+"/"+country
    country_path_res = res_path+"/"+country
    out_path_country = out_path+"/"+country
    createFolder(out_path_country)
    
    ## Creating dataframe for test data
    test_data = os.listdir(test_data_path)
    new_test_data = []
    for i in range(len(test_data)):
        name = test_data[i].split('.')[0]
        name_update = '_'.join(name.split('_')[:-1])
        country_name = name.split('_')[-1]
#         print(country,country_name)
        if country == country_name:
            new_test_data.append(name_update)
    df_test = pd.DataFrame()
    df_test['name'] = new_test_data
#     print(df_test)
    df_org = pd.read_csv(country_path_res+".csv")
    
    ## Keep only test data record
    df = df_org[df_org['name'].isin(df_test['name'])]
    df = df.reset_index(drop = 'index')
    print(len(df))
    
    df_total = df_total.append(df)
    print(len(df_total))
    
    img_types = os.listdir(country_path_heatmap)
    print("##### "+country+" #####")
    for img_type in img_types:
        out_path_img_type = out_path_country+'/'+img_type
        country_path_heatmap_type = country_path_heatmap+'/'+img_type
        createFolder(out_path_img_type)
        print("---- "+img_type+" ----")
        df_type = df[['name','gender',img_type]]
#         print(df_type)
        df_male = df_type[df_type['gender'] == 'M']
        df_female = df_type[df_type['gender'] == 'F']
#         print(len(df_male))
#         print(len(df_female))
        df_male_correct = df_male[df_male[img_type] == 'M']
        df_male_wrong = df_male[df_male[img_type] != 'M']
        
        df_female_correct = df_female[df_female[img_type] == 'F']
        df_female_wrong = df_female[df_female[img_type] != 'F']
        print(len(df_male_correct),len(df_male_wrong),len(df_female_correct),len(df_female_wrong))
        
        correct_path = out_path_img_type+'/correct'
        wrong_path = out_path_img_type+'/wrong'
        createFolder(correct_path)
        createFolder(wrong_path)
        correct_path_male = correct_path+'/male'
        correct_path_female = correct_path+'/female'
        createFolder(correct_path_male)
        createFolder(correct_path_female)
        
        wrong_path_male = wrong_path+'/male'
        wrong_path_female = wrong_path+'/female'
        createFolder(wrong_path_male)
        createFolder(wrong_path_female)
        
        ## copy correct male
        for name in df_male_correct['name']:
            src = country_path_heatmap_type+'/'+name+'.jpg'
            dst = correct_path_male+'/'+name+'.jpg'
            try:
                shutil.copy(src,dst)
            except:
                pass
        
        ## copy correct female
        for name in df_female_correct['name']:
            src = country_path_heatmap_type+'/'+name+'.jpg'
            dst = correct_path_female+'/'+name+'.jpg'
            try:
                shutil.copy(src,dst)
            except:
                pass
        
        ## copy wrong male
        for name in df_male_wrong['name']:
            src = country_path_heatmap_type+'/'+name+'.jpg'
            dst = wrong_path_male+'/'+name+'.jpg'
            try:
                shutil.copy(src,dst)
            except:
                pass
        
        ## copy wrong female
        for name in df_female_wrong['name']:
            src = country_path_heatmap_type+'/'+name+'.jpg'
            dst = wrong_path_female+'/'+name+'.jpg'
            try:
                shutil.copy(src,dst)
            except:
                pass

60
60
##### Australia #####
---- Greyish ----
24 16 11 9
---- masked ----
5 35 3 17
---- normal ----
23 17 13 7
---- RGB0.3 Noise ----
14 26 1 19
---- RGB0.5 Noise ----
3 37 0 20
---- Spread Noise ----
18 22 5 15
60
120
##### Bangladesh #####
---- Greyish ----
22 18 12 8
---- masked ----
6 34 5 15
---- normal ----
19 21 9 11
---- RGB0.3 Noise ----
9 31 4 16
---- RGB0.5 Noise ----
2 38 0 20
---- Spread Noise ----
22 18 4 16
60
180
##### England #####
---- Greyish ----
18 22 13 7
---- masked ----
1 39 3 17
---- normal ----
14 26 12 8
---- RGB0.3 Noise ----
6 34 2 18
---- RGB0.5 Noise ----
2 38 0 20
---- Spread Noise ----
15 25 9 11
60
240
##### India #####
---- Greyish ----
19 21 10 10
---- masked ----
7 33 6 14
---- normal ----
18 22 10 10
---- RGB0.3 Noise ----
10 30 1 19
---- RGB0.5 Noise ----
2 38 0 20
---- Spread Noise ----
18 22 3 17
60
300
##### New Zealand #####
---- Greyish ----
18 22 6 14
---- masked ----
6 34 0 20
---- normal ----
19 21 6 14
---- RGB0.3 Noise ----
10 30 1 19
-

In [55]:
print("Original faces not detected")
total_male_detected = len(df_total[df_total['normal'] == 'M'])
total_female_detected = len(df_total[df_total['normal'] == 'F'])
print(len(df_total)-total_male_detected-total_female_detected)

Original faces not detected
209


## Number of images not detected in zero shot

In [79]:
import numpy as np
path = "./../Results in csv files/deepface/"
countries = os.listdir(path)
total_img = 0
total = 0
for country in countries:
    print("country : ",country.split('.')[0])
    path_country = path+country
    df = pd.read_csv(path_country)
    df = df[['name','gender','normal','valid normal']]
    df = df[df['valid normal'] == 1]
    print(len(df))
    total_img += len(df)
    df = df.dropna(subset=['normal'])
    print(len(df))
    total += len(df)

country :  Australia
941
517
country :  Bangladesh
367
207
country :  England
1418
521
country :  India
2254
1170
country :  New Zealand
597
279
country :  Pakistan
557
335
country :  South Africa
541
305
country :  West Indies
649
288


In [80]:
total_img

7324

In [81]:
total

3622